#Project: Personal Health Tracker
Module: Fusion-Ready LSTM for ECG Signals

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

#Load CSVs
train_df = pd.read_csv('/content/ECG/mitbih_train.csv', header=None)
test_df  = pd.read_csv('/content/ECG/mitbih_test.csv', header=None)

#Features & Labels
X_train = train_df.iloc[:, :-1].values
y_train = train_df.iloc[:, -1].values

X_test  = test_df.iloc[:, :-1].values
y_test  = test_df.iloc[:, -1].values

# Normalize ECG signals
X_train = X_train / X_train.max()
X_test  = X_test / X_test.max()

# Map multi-class labels to binary

y_train = np.where(y_train == 0, 0, 1)
y_test  = np.where(y_test == 0, 0, 1)

# Reshape for LSTM
X_train = X_train[:, :, None]
X_test  = X_test[:, :, None]

#Validation split
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Fusion-Ready LSTM
timesteps = X_train_final.shape[1]
features  = 1

lstm_input = Input(shape=(timesteps, features), name="lstm_input")

# Bidirectional + stacked LSTM
x = Bidirectional(LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001)))(lstm_input)
x = Dropout(0.2)(x)
x = LSTM(32, kernel_regularizer=l2(0.001))(x)
x = Dropout(0.2)(x)

# Dense embedding for fusion
lstm_embedding = Dense(64, activation='relu', name="lstm_embedding")(x)

# Output layer
output = Dense(1, activation='sigmoid', name="lstm_output")(lstm_embedding)

# Build model
lstm_model = Model(inputs=lstm_input, outputs=output, name="fusion_ready_lstm")
lstm_model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.summary()

#Train with Early Stopping
es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = lstm_model.fit(
    X_train_final, y_train_final,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[es]
)

#Evaluate on Test
test_loss, test_acc = lstm_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc*100:.2f}%")
lstm_model.save("fusion_ready_lstm_model.h5")

#Save Embedding Model for Fusion
embedding_model = Model(inputs=lstm_model.input, outputs=lstm_embedding)
embedding_model.save("fusion_ready_lstm_embedding_model.h5")
print("LSTM embedding model saved. Ready for fusion with CNN + ANN!")


Model: "fusion_ready_lstm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_input (InputLayer)         │ (None, 187, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 187, 128)       │        33,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 187, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        20,608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_embedding (Dense)          │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_output (Dense)             │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 56,577 (221.00 KB)

 Trainable params: 56,577 (221.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 490s 222ms/step - accuracy: 0.8263 - loss: 0.5702 - val_accuracy: 0.8277 - val_loss: 0.4734
Epoch 2/20
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 506s 224ms/step - accuracy: 0.8263 - loss: 0.4752 - val_accuracy: 0.8277 - val_loss: 0.4657
Epoch 3/20
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 470s 215ms/step - accuracy: 0.8294 - loss: 0.4637 - val_accuracy: 0.8277 - val_loss: 0.4635
Epoch 4/20
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 475s 217ms/step - accuracy: 0.8273 - loss: 0.4646 - val_accuracy: 0.8277 - val_loss: 0.4628
Epoch 5/20
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 498s 216ms/step - accuracy: 0.8286 - loss: 0.4611 - val_accuracy: 0.8277 - val_loss: 0.4606
Epoch 6/20
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 484s 221ms/step - accuracy: 0.8288 - loss: 0.4601 - val_accuracy: 0.8277 - val_loss: 0.4604
Epoch 7/20
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 487s 214ms/step - accuracy: 0.8267 - loss: 0.4627 - val_accuracy: 0.8277 - val_loss: 0.4602
Epoch 8/20
2189/2189 ━━━━━━━━━━━━━━━━━━━━ 466s 213ms/step - ac

✅ Test Accuracy: 82.76%


✅ LSTM embedding model saved. Ready for fusion with CNN + ANN!
